In [1]:
# --- Imports and Setup ---
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sqlite3
import os

In [2]:
# --- Configuration ---
DB_PATH = '../token_holdings.db'

In [3]:
# Check if the database file exists
if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f"Database not found at '{DB_PATH}'. Please run `make production` first.")

In [4]:
# --- Connect to the database ---
conn = sqlite3.connect(DB_PATH)

--- Chart 1: Normalized Distribution of Token Holders by Percentile ---

In [5]:
# Query the data
df_percentiles = pd.read_sql_query("""
    SELECT
        token_symbol,
        percentile_rank,
        balance
    FROM token_holdings
    WHERE is_percentile_marker = 1
    ORDER BY token_symbol, percentile_rank DESC;
""", conn)

In [6]:
# Create the plot
fig1 = go.Figure()

In [7]:
for token in df_percentiles['token_symbol'].unique():
    token_df = df_percentiles[df_percentiles['token_symbol'] == token]
    
    # Normalize the balance by the max balance for that token
    normalized_balance = token_df['balance'] / token_df['balance'].max()
    
    fig1.add_trace(go.Scatter(
        x=token_df['percentile_rank'],
        y=normalized_balance,
        mode='lines+markers',
        name=token
    ))

In [8]:
fig1.update_layout(
    title='Normalized Holder Balance by Percentile',
    xaxis_title='Percentile Rank',
    yaxis_title='Normalized Balance (as % of 99th percentile)',
    yaxis_tickformat='.2%',
    hovermode='x unified',
    legend_title='Token'
)

In [9]:
fig1.show()

--- Chart 2: Top 10 Holders by Token Balance ---

In [10]:
# Query the data
df_top_balance = pd.read_sql_query("""
    SELECT
        token_symbol,
        account_id,
        balance,
        balance_rank
    FROM token_holdings
    WHERE is_top_holder = 1
    ORDER BY token_symbol, balance_rank;
""", conn)

# Create the plot
fig2 = px.bar(
    df_top_balance,
    x='balance_rank',
    y='balance',
    color='token_symbol',
    facet_col='token_symbol',
    facet_col_wrap=1,
    height=800,
    labels={'balance_rank': 'Holder Rank', 'balance': 'Token Balance'},
    title='Top 10 Holders by Token Balance Distribution',
    hover_data=['account_id'] # Show account_id on hover
)

fig2.update_yaxes(matches=None) # Allow y-axes to have different scales
fig2.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig2.update_xaxes(tickvals=list(range(1, 11))) # Ensure x-axis shows ranks 1-10
fig2.show()

--- Chart 3: Top 10 Holders by USD Value ---

In [11]:
# Query the data
df_top_usd = pd.read_sql_query("""
    SELECT
        token_symbol,
        account_id,
        usd_value
    FROM token_holdings
    WHERE is_top_holder = 1 AND usd_value IS NOT NULL
    ORDER BY token_symbol, usd_value DESC;
""", conn)

# Create the plot
if df_top_usd.empty:
    print("No data with USD values found. Ensure a refresh was run with USD features enabled.")
else:
    # Create a rank within each token group
    df_top_usd['usd_rank'] = df_top_usd.groupby('token_symbol')['usd_value'].rank(method='first', ascending=False).astype(int)

    fig3 = px.bar(
        df_top_usd,
        x='usd_rank',
        y='usd_value',
        color='token_symbol',
        facet_col='token_symbol',
        facet_col_wrap=1,
        height=800,
        labels={'usd_rank': 'Holder Rank', 'usd_value': 'USD Value'},
        title='Top 10 Holders by USD Value Distribution',
        hover_data=['account_id'] # Show account_id on hover
    )
    
    fig3.update_yaxes(matches=None) # Allow y-axes to have different scales
    fig3.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
    fig3.update_xaxes(tickvals=list(range(1, 11))) # Ensure x-axis shows ranks 1-10
    fig3.show()

In [12]:
# Close the database connection
conn.close()